In [1]:
# Created by trilo at 15-01-2024

from langchain.llms import GooglePalm
from langchain.utilities import SQLDatabase
# from langchain_community.llms import GooglePalm
# from langchain_community.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

## Google LLM API key and connection

In [2]:
api_key = 'YOUR_API_KEY'
llm = GooglePalm(google_api_key=api_key, temperature=0.2)
# print(llm("write poem on aaloo dhosa"))

C:\Users\trilo\anaconda3\envs\llms\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Database credentials and connection

In [3]:
db_user = "root"
db_password = "root"
db_host = "localhost"
db_name = "atliq_tshirts"
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",
                          sample_rows_in_table_info=3)
# print(db.table_info)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

## testing query

In [4]:
qns1 = db_chain.run("How many t-shirts do we have left for levis in extra small size and white color?")
qns1



> Entering new SQLDatabaseChain chain...
How many t-shirts do we have left for levis in extra small size and white color?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Levi' AND color = 'White' AND size = 'XS'
SQLResult: [(64,)]
Answer:64
> Finished chain.


'64'

In [5]:
# getting the wrong answer with this query
# qns2 = db_chain.run("What is the total price of the inventory for all small size t-shirts?")
# actual query
qns2 = db_chain.run("SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S';")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S';
SQLQuery:SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('18320'),)]
Answer:18320
> Finished chain.


In [6]:
# getting error
# qns3 = db_chain.run("If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our "
#                     "store will generate (post discounts)?")
# actual query
sql_code = """
select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """
qns3 = db_chain.run(sql_code)



> Entering new SQLDatabaseChain chain...

select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 
SQLQuery:select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('24410.950000'),)]
Answer:24410.95
> Finished chain.


In [7]:
# getting wrong output
# qns4 = db_chain.run("What is the total price of the inventory for levi t-shirts?")
# actual SQL query
qns4 = db_chain.run("SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(Decimal('26941'),)]
Answer:26941
> Finished chain.


In [8]:
# getting wrong outout
# qns5 = db_chain.run("How many white color Levi's t shirts we have available?")
# actual SQL query
qns5 = db_chain.run("SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'")



> Entering new SQLDatabaseChain chain...
SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('178'),)]
Answer:178
> Finished chain.


### Few shot learning

In [9]:
few_shots = [
    {'Question': "How many t-shirts do we have left for levis in extra small size and white color?",
     'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
     'SQLResult': "Result of the SQL query",
     'Answer': qns1},
    {'Question': "How much is the total price of the inventory for all S-size t-shirts?",
     'SQLQuery': "SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
     'SQLResult': "Result of the SQL query",
     'Answer': qns2},
    {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our "
                 "store will generate (post discounts)?",
     'SQLQuery': "SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from "
     "(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi' group by "
     "t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id",
     'SQLResult': "Result of the SQL query",
     'Answer': qns3},
    {'Question': "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate "
                 "without discount?",
     'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
     'SQLResult': "Result of the SQL query",
     'Answer': qns4},
    {'Question': "How many white color Levi's t-shirts we have available?",
     'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer': qns5
     }
     ]

### Hugging Face Embeddings

In [10]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

to_vectorize = [" ".join(example.values()) for example in few_shots]

In [11]:
to_vectorize

["How many t-shirts do we have left for levis in extra small size and white color? SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS' Result of the SQL query 64",
 "How much is the total price of the inventory for all S-size t-shirts? SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S' Result of the SQL query 18320",
 "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)? SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from (select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi' group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id Result of the SQL query 24410.95",
 "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount? SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = '

### Chroma for vector store

In [12]:
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=few_shots)

In [14]:
vectorstore

In [15]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

example_selector.select_examples({"Question": "How many Adidas T shirts I have left in my store?"})

[{'Answer': '64',
  'Question': 'How many t-shirts do we have left for levis in extra small size and white color?',
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
  'SQLResult': 'Result of the SQL query'},
 {'Answer': '178',
  'Question': "How many white color Levi's t-shirts we have available?",
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
  'SQLResult': 'Result of the SQL query'}]

### Prompt Engineering 

In [16]:
### my sql based instruction prompt
mysql_prompt = """You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: Query to run with no pre-amble
SQLResult: Result of the SQLQuery
Answer: Final answer here

No pre-amble.
"""

In [17]:
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


In [20]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [21]:
print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [22]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"], #These variables are used in the prefix and suffix
)

In [23]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)

In [30]:
new_chain("How many total tshirt with white colour from brand levi")



> Entering new SQLDatabaseChain chain...
How many total tshirt with white colour from brand levi
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('178'),)]
Answer:178
> Finished chain.


{'query': 'How many total tshirt with white colour from brand levi',
 'result': '178'}

## Testing

In [32]:
new_chain("How much is the price of the inventory for all extra large size t-shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all extra large size t-shirts?
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE size = 'XL'
SQLResult: [(Decimal('16912'),)]
Answer:16912
> Finished chain.


{'query': 'How much is the price of the inventory for all extra large size t-shirts?',
 'result': '16912'}

In [34]:
new_chain("How much is the price of all black color levi t shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of all black color levi t shirts?
SQLQuery:SELECT sum(price*stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'Black'
SQLResult: [(Decimal('6716'),)]
Answer:6716
> Finished chain.


{'query': 'How much is the price of all black color levi t shirts?',
 'result': '6716'}

In [36]:
new_chain("If we have to sell all the levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?
SQLQuery:SELECT sum(price * stock_quantity * ((100-COALESCE(discounts.pct_discount,0))/100)) FROM t_shirts LEFT JOIN discounts ON t_shirts.t_shirt_id = discounts.t_shirt_id WHERE brand = 'Levi'
SQLResult: [(Decimal('24410.950000'),)]
Answer:24410.95
> Finished chain.


{'query': 'If we have to sell all the levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?',
 'result': '24410.95'}

In [39]:
new_chain.run('How much revenue  our store will generate by selling all Van Heuson TShirts without discount?')



> Entering new SQLDatabaseChain chain...
How much revenue  our store will generate by selling all Van Heuson TShirts without discount?
SQLQuery:SELECT sum(price * stock_quantity) FROM t_shirts WHERE brand = 'Van Huesen'
SQLResult: [(Decimal('24936'),)]
Answer:24936
> Finished chain.


'24936'